In [ ]:
import os
if not os.path.exists('/content/police-records-project'):
    !git clone https://github.com/c-goenka/police-records.git
    %cd your-ml-project
    !pip install -r requirements.txt
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import fitz
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

In [ ]:
def extract_text(file_path):
  full_text = ''
  doc = fitz.open(file_path)

  for i, page in enumerate(doc):
    text = page.get_text()
    if len(text.strip()) < 50:
      images = convert_from_path(file_path, first_page=i+1, last_page=i+1, dpi=300)
      if images:
        page_image = images[0]
        text = pytesseract.image_to_string(page_image)
    full_text += text + "\n"

  return full_text

In [ ]:
data_path = "/content/MyDrive/police-records-project-data"
records = []

for label in os.listdir(data_root):
    folder_path = os.path.join(data_root, label)
    if os.path.isdir(folder_path):
        print(f"Processing category: {label}")
        for filename in os.listdir(folder_path):
            if filename.endswith(".pdf"):
                file_path = os.path.join(folder_path, filename)
                extracted_text = extract_text_robust(file_path)

                # Truncation for token limits (First 300 + Last 200 words)
                words = extracted_text.split()
                if len(words) > 500:
                    truncated = " ".join(words[:300]) + " [SEP] " + " ".join(words[-200:])
                else:
                    truncated = extracted_text

                records.append({
                    "filename": filename,
                    "text": truncated,
                    "label": label
                })

In [ ]:
df = pd.DataFrame(records)
df.to_csv("police_docs_master.csv", index=False)
print("Data saved to police_docs_master.csv")